In [ ]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

# Lecture 04: Workflow Communication Pattern

This notebook demonstrates the workflow pattern where a single tool orchestrates multiple specialized agents in sequence. The stock_report tool uses a multi-agent workflow:
1. **Agent 1 (Analysis)**: Gathers stock information
2. **Agent 2 (Recommendation)**: Generates investment recommendations


In [ ]:
# Setup: Import necessary modules
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext
from pydantic_ai.tools import Tool
from common.models import StockReport
from common.tools import web_search_tool
from common.utils import create_agent_model, setup_logging, get_current_date

# Initialize logging
setup_logging()

In [ ]:
# Add Logfire setup
from common.utils import setup_logfire

setup_logfire(
    service_name="workflow-pattern-lecture04",
    start_message="🚀 Lecture 04 - Workflow Pattern Notebook Started",
)

In [ ]:
# Create intermediate model for passing data between agents
class StockAnalysis(BaseModel):
    """Stock analysis without recommendation - output from Agent 1."""

    symbol: str
    company_name: str
    current_price: float | None = None
    executive_summary: str
    recent_news: list[str] = Field(default_factory=list)
    financial_highlights: list[str] = Field(default_factory=list)
    market_sentiment: str
    risk_factors: list[str] = Field(default_factory=list)
    analysis_date: str


class StockRecommendationOnly(BaseModel):
    """Investment recommendation - output from Agent 2."""

    recommendation: str


# Agent 1: Gathers stock information (no recommendation)
stock_analysis_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool],
    output_type=StockAnalysis,
    system_prompt="You are a financial analyst. Gather comprehensive stock information WITHOUT making recommendations.",
)

# Agent 2: Generates recommendations based on analysis
recommendation_agent = Agent(
    model=create_agent_model(),
    output_type=StockRecommendationOnly,
    system_prompt="You are an investment advisor. Generate recommendations based on provided analysis data.",
)


# Create workflow tool that orchestrates both agents
async def stock_report(ctx: RunContext, symbol: str) -> StockReport:
    """Multi-agent workflow: Agent 1 analyzes, Agent 2 recommends."""
    # Step 1: Run analysis agent
    analysis_prompt = f"Analyze {symbol}. Analysis date: {get_current_date()}"
    analysis_result = await stock_analysis_agent.run(analysis_prompt)
    stock_data = analysis_result.output

    # Step 2: Run recommendation agent with analysis data
    rec_prompt = f"""Based on this analysis for {symbol}:
    
Company: {stock_data.company_name}
Summary: {stock_data.executive_summary}
Sentiment: {stock_data.market_sentiment}
Risks: {', '.join(stock_data.risk_factors)}

Provide your investment recommendation."""

    rec_result = await recommendation_agent.run(rec_prompt)

    # Step 3: Combine results into final report
    return StockReport(
        symbol=stock_data.symbol,
        company_name=stock_data.company_name,
        current_price=stock_data.current_price,
        executive_summary=stock_data.executive_summary,
        recent_news=stock_data.recent_news,
        financial_highlights=stock_data.financial_highlights,
        market_sentiment=stock_data.market_sentiment,
        risk_factors=stock_data.risk_factors,
        recommendation=rec_result.output.recommendation,
        analysis_date=stock_data.analysis_date,
    )


workflow_stock_tool = Tool(
    stock_report, description="Get stock analysis via multi-agent workflow"
)

# Create financial assistant with workflow tool
workflow_financial_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool, workflow_stock_tool],
    system_prompt="You are a financial assistant. Use stock_report for detailed stock analysis (multi-agent workflow).",
)

In [ ]:
# Ask a general financial question
question = "What is the current market sentiment?"
result = await workflow_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

In [ ]:
# Ask for detailed stock analysis (triggers workflow)
# This will:
# 1. Use stock_analysis_agent to gather information
# 2. Use recommendation_agent to generate recommendations
# 3. Combine results into complete StockReport
question = "Can you give me a detailed analysis of Microsoft stock (MSFT)?"
result = await workflow_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

In [ ]:
# Try another stock analysis with workflow
question = "Give me a comprehensive report on NVIDIA (NVDA)"
result = await workflow_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")